# Julia 3: Choices, Loops, Functions, and Performance issues
### Bernt Lie
#### University of South-Eastern Norway
#### May, 2021; minor update September 2023

## Learning goals
In this session, we will look at structures for automation:
* conditional choices
* loops
* functions
* performance tips

## Conditional choices
### General if statement
The standard `if` syntax is:
```julia
if condition_1
    <block_1>
elseif condition_2
    <block_2>
    ...
elseif condition_n
    <block_n>
else
    <block_n+1>
end
```
Here, a `condition_j` is a statement having a `Bool` outcome, and may be composed of Bool variables, relational expressions, logical expressions, etc. Depending on the conditions, the first condition (`condition_j`) that is `true` is chosen, `block_j` is executed, and the if-statement is finished.

As in most computer languages, there may be one condition, or many.

### Ternary operator

A ternary operator `a ? b : c` exists as a short form of:
```julia
if a
    b
else
    c
end
```
This ternary operator is typically used in single line tests. 

In [1]:
x = 3
if x > 3
    println("Yes, greater")
else
    println("No, not greater")
end

No, not greater


In [2]:
x = 3
if x > 3
    println("Yes, greater")
elseif x == 3
    println("Equal to")
else
    println("No, not greater")
end

Equal to


## Loops
### For loops
#### Basic loop
The general syntax is
```julia
for c in col
    <block>
end
```
Here, `col` is some collection or iterator, and `c` will in turn be given a value by traversing `col`. Alternatively to `for c in col`, the syntax `for c = col` is valid.

As in all languages, it is possible to have loops within loops. In Julia, there is a simplified syntax for this:
```julia
for i in I, j in J, ...
    <block>
end
```
which is a short form of
```julia
for i in I
    for j in J
        ...
        <block>
        ...
    end
end
```

#### Enumeration
Sometimes, both the *value* and the *index* in the collection is needed:
```julia
for (i,c) in enumerate(col)
    <block>
end
```

#### Comprehension
Comprehension is an efficient way to create arrays without intermediate arrays. The syntax is:
```julia
[expression for c in C]
```
Here, `expression` is typically computed from `c`, and one vector element is created for each `c` in `C`. The syntax can be extended with more loops, leading to higher dimensional arrays:
```julia
[expression for c1 in C1, c2 in C2, ...]
```

#### Variable scope and loops
Iterators in loops (e.g., `c` in `for c in C`) are local to the loop, and its value does not exist outside of the loop.

Perhaps more unusual is that variables defined inside a loop have local scope within the loop, and do not exist when the loops is finished. However, if a variable has been defined prior to a loop, and this variable is modified inside the loop, then the modification exists after the loop.

In [3]:
for i in 1:5
    println("Counter i = $i")
end

Counter i = 1
Counter i = 2
Counter i = 3
Counter i = 4
Counter i = 5


In [4]:
D = Dict([(:a,2),(:b,3),(:c,4)])
for d in D
    println("Dictionary elements $d, key $(d[1]), value $(d[2])")
end

Dictionary elements :a => 2, key a, value 2
Dictionary elements :b => 3, key b, value 3
Dictionary elements :c => 4, key c, value 4


In [5]:
for i in 1:2, j in 1:2
    println("Result for ($i,$j) is $(i^2-j)")
end

Result for (1,1) is 0
Result for (1,2) is -1
Result for (2,1) is 3
Result for (2,2) is 2


In [6]:
A = rand(-9:9,5)
#
for (i,v) in enumerate(A)
    println("Element $i has value $v")
end

Element 1 has value 4
Element 2 has value -4
Element 3 has value 0
Element 4 has value 4
Element 5 has value -4


In [7]:
A = [i^2 for i in 1:3]

3-element Vector{Int64}:
 1
 4
 9

In [8]:
A = [(i^2-j) for i in 1:2, j in 1:3]

2×3 Matrix{Int64}:
 0  -1  -2
 3   2   1

In [9]:
# variable `k` does not exist prior to the loop
for i in 1:3
    k = i
    println("$k")
end

1
2
3


In [10]:
k    # variable `k` was only created locally within the loop, and does not have
     # a value outside of the loop

UndefVarError: UndefVarError: `k` not defined

In [11]:
k = 0    # Here, variable `k` is defined before the loop is executed. Variable 
         # `k` by default gets *global scope*.
for i in 1:3
    k = i
    println("$k")
end

1
2
3


In [12]:
k    # Because `k` had global scope prior to the loop, the loop changes the 
     # value of a variable which is accessible outside of the loop

3

#### While loop
The `while` loop is used when we do not know in advance how many times we need to go through the loop. Typical examples are within numeric methods where we do not know in advance how many iterations are required to reach a certain accuracy goal. The structure is as in most languages:
```julia
while boolvar
    statements
end
```
The statements are executed as long as the Bool variable/expression `boolvar` is `true`. To avoid an infinite loop, the `statements` must contain some provision to change `boolvar` to `false` under given conditions.

In [13]:
x = 0
cont = true
while cont
    x += 1
    if x > 10
        cont = false
    end
end
x

11

## Blocks
Sometimes, it is convenient to group code in blocks. Julia provides the following two structures for blocks: `begin ... end` and `let ... end`.
### Grouping statements
The basic syntax for the `begin ... end` statement is:
```julia
begin 
    statements
end
```
The key use of this construct is in combination with other statements where only a single line should be used. Examples could be in "one line" function definitions (see below), in the `Pluto` notebook system, etc. 

Observe that all variables defined within the `begin ... end` block are available in the scope where the block is defined.

### Grouping statements with local variables
The basic syntax of the `let ... end` statement is:
```julia
let
    statements
end
```
For the `let` block, it is also possible to add variable declarations on the first line, and optionally to give them values on this line:
```julia
let var1 = 2, var2, var3 = 5
    statements
end
```
The *key* difference with the `begin` block is that the variables within a `let` block are *local*, i.e., they do not have a value outside of the scope of the block. Only the result on the last line of the block is available outside of the block. The last line can, e.g., an assignment, or it can be a `return` statement which explicitly declares what to return out of the block.

When it comes to scope of variables, the `let` block is similar to *functions*, see below.

In [14]:
begin 
    u = 3
    w = u+2
end

5

In [15]:
u, w    # Observe that all variables defined in the block have values outside of the block

(3, 5)

In [16]:
let
    m = 3
    n = m+2    
end

5

In [17]:
n

UndefVarError: UndefVarError: `n` not defined

In [18]:
let 
    p = 3
    q = p+2
    return p, q
end

(3, 5)

## Functions
### General functions
The basic syntax for defining a function in Julia is:
```julia
function f(x)
    statements
    return result
end
```
Here, `x` is the argument of the function. The statement `return result` determines what is returned from the function; typically `result` is a tuple, e.g., specified by a comma separated list of elements. If statement `return` is dropped, the last result computed in the `statements` is returned.

Any variable introduced inside of the function has *local scope*, and can not be examined outside of the function.

Functions can have multiple input arguments. Arguments are given *positional* reference (`args`) or are referenced by keywords. Arguments are separated by comma as in tuples. However, to make a distinction between positional arguments and keyword arguments, a semicolon is used:
```julia
function f(arg1, arg2, ..., argn; kwarg1, krarg2,...)
    statements
end
```
It is possible to give default values to both positional and keyword arguments, e.g.:
```julia
function f(arg1, arg2, arg3=val1,...; kwarg1 = kwval1,...)
    statements
end
```

When calling a function, the positional arguments (those occurring before ";") must be given in the specified order. However, keyword arguments can be given in any order. Example:
```julia
function f(x,n=1; a=1, b=2, c=1)
    return a*x^n + b*(1/x)^n + c*log(n+1,x)
end
```
This function can be called as `f(2)`, `f(2,3)`, `f(2;a=5)`, `f(2;c=3,a=2)`, etc. The point with keyword arguments is that they can be given in arbitrary order. Positional variables, however, *have* to be given in the specified order.

A final comment: in the *definition* of functions, a semicolon must be used to separate positional arguments vs. keyword arguments. However, when *calling* a function, use of semicolon is optional: we can also just use comma when calling the function.

### Type declaration for arguments

It is possible to specify the type of arguments using the syntax `x::T` where `T` is some type. As an example:
```julia
function f(x::Number, n::Int=1; a::Number=1, b::Number=2, c::Number = 1)
    statements
end
```
will restrict the type of allowed arguments, and may help in avoiding erroneus use of  function. Think of the `x :: T` as in mathematics: $x \in \cal{T}$ where $\cal{T}$ is a set. **However**, it is recommended to use as few restrictions as possible on the type of function arguments.

### Function dispatching

A key concept in Julia is that of function *dispatching*. The idea is that it is possible to define many different "functions" in Julia with the same name, but with different number of arguments, and with different type restrictions on arguments. These different versions of functions with the same name are referreded to as *methods*. [In many cases, dispatching is the same as function/operator overloading, but not in all cases.] Dispatching solves one problem with namespace in other languages: in, e.g., Python, if you write a new function, and accidentally give it the same name as an existing function, you simply overwrite the existing function so that it is not available any more. This is so no matter what arguments the function has, and what its results are. In Julia, on the other hand, it is possible to create functions with the same name as existing ones, without destroying the functionality of the original functions -- as long as the two functions do not have identical arguments (including: identical types).

A simple example of dispatching: the Julia function `range` can be used to produce an iterator of numbers, as in `range(0,2*pi,length=10)`. But if the two first arguments are *colors*, the the function produces an iterator of *colors* between the two given values; we'll see examples of this later.

### One-line functions
Simple functions used in mathematics, etc., can be given a simplified on-line defintion with syntax:
```julia
f(x) = statement
```
Such functions can also have multiple arguments, keyword arguments, etc. If the `statement` is too complicated to fit in one-line, it can be wrapped in a `begin... end` block, and stretch over multiple lines:
```julia
f(x) = begin
    statement_1
    statement_2
    ...
end
```

### Anonymous functions
Anonymous functions, or lambda functions, have the syntax `x -> statement`. It is possible to use several arguments: `(x,y,...) -> statement`. These anonymous functions are typically used in function calls when one does not want to create named functions.

It is also possible to give name to anonymous functions and achieve something similar to in one-line functions, e.g.,
```julia
f = x -> statement
```
whereupon the function is called as `f(x)`.

### Chaining of functions
Two functions, `f(x)` and `g(x)` can be chained provided as `g(f(x))` provided that the return value of `f(x)` matches the required argument of `g(x)`. 

Such a chaining can also be written using syntax: `g∘f(x)` where `∘` is introduced as `\circ + TAB`.

Furthermore, such a chaining can also be written using *function piping*, with notation `f(x) |> g`.

With all three ways of achievng function chaining, the requirement on return value of `f(x)` to match the required argument of `g(x)` is the same. 

In the case that the returned value of `f(x)` does not match the required argument of `g(x)`, it is convenient to use anonymous functions to correct this. Such corrections may get somewhat cluttered, and to reduce the clutter, piping is recommended.

### Unpacking collections to sequence of arguments
Suppose we have a collection that we need to pass to a function where the function requires multiple arguments. We can then *unpack* the collection `C` as follows: `f(C...)`.

In [19]:
function f(x::Number,n::Int=1;a::Number=1,b::Number=2,c::Number=1)
    return a*x^n + b*(1/x)^2 + c*log(n+1,x)
end

f (generic function with 2 methods)

In [20]:
f(3)

4.807184722943378

In [21]:
f(3,2)

10.222222222222221

In [22]:
f(3;c=2)

6.392147223664535

In [23]:
myfun(x) = sin(x)*exp(-x/3)    # One-line function

myfun (generic function with 1 method)

In [24]:
myfun(pi/3)

0.6108481445504094

In [25]:
myfun1 = x -> sin(x)*exp(-x/3)    # Giving name `myfun1` to anonymous function

#16 (generic function with 1 method)

In [26]:
myfun1(pi/3)

0.6108481445504094

In [27]:
sin(cos(pi/3))

0.4794255386042031

In [28]:
(sin∘cos)(pi/3)

0.4794255386042031

In [29]:
cos(pi/3) |> sin

0.4794255386042031

In [30]:
cos(pi/3) |> x-> log(7,x)    # Example on how to pipe the result of 
                             # a function with single result into 
                             # function requiring 2 arguments

-0.35620718710802207

In [31]:
v = rand(-9:9,5)

5-element Vector{Int64}:
 -4
 -7
 -8
 -3
  8

In [32]:
max(v...)    # Unpacking vector into a comma separated sequence of scalars

8

## User-defined types
Julia types are dynamic, nominative and parametric. The `::` operator can be used to attach type annotations to expressions and variables in programs. This may add extra information to the (Just-In-Time; JIT) compiler so that the compiled code runs more efficiently, but it may also help to inform the user about a wrong choice of argument in function calls.

### Abstract types
Abstract types are defined using the syntax `abstract type «name» end` or `abstract type «name» <: «supertype» end`. An abstract type is used as the "main branch" of several more specialized abstract types, or as the main branch of several concrete types. In Julia, the most general type is `Any`. One example of an abstract numeric type is `Number`; another is `Real`. This means that Julia has built-in the following abstract types:
```julia
abstract type Number end
abstract type Real <: Number end
```
and so on. Suppose a user wants to define a new type for dynamic systems. This could be, e.g., `abstract type DynamicSystem end`. Then perhaps one wants to create concrete subtype named `LTI` for Linear, Time Invariant system. To do that, we need to discuss concrete, composite types.

### Primitive types
The primitive types are built-in types, which are defined as, e.g., `primitive type Int16 <: Signed 16 end`, which essentially says that type `Int16` is a subtype of abstract type `Signed`, and takes up `16 bits` of memory space.

### Composite types
Composite types are denoted "records", "structs", or "objects" in other languages. Immutable types are defined in a `struct`... `end` block, where the block defines the data fields of the type.

```julia
struct typename
    field specification block
end
``` 
The field specification block lists the field names, and optionally their type. If field names are given without type, they are assumed to be of type `Any`. The type name also works as a type *constructor*/instantiator, which is a function with the same number of arguments as there are fields.

Suppose we want to define a type for storing a circle with 2D position, radius, and color via RGB values. This can be done via:

In [33]:
struct Circle
    x::Int
    y::Int
    r::Int
    R::Int
    G::Int
    B::Int
end

In [34]:
mycirc = Circle(10,20,5,0,0,0)

Circle(10, 20, 5, 0, 0, 0)

We can query the field names of the type using function `fieldnames(type)`. We can access the values of the various fields of the *instance* using dot-notation:

In [35]:
fieldnames(Circle)    # Observe that we query the field names of 
                      # the *type*/*struct* name. The result is a 
                      # simple symbol, i.e., the "unevaluated" field names

(:x, :y, :r, :R, :G, :B)

In [36]:
mycirc.x    # Observe that we request the field *value* of 
            # the *instantiated struct* using dot-notation where 
            # the field name is *without* the "unevaluation" symbol ":"

10

In [37]:
mycirc.R

0

The standard struct is *immutable*, in that once populated, it is not possible to change the values of the fields:

In [38]:
mycirc.R = 10

ErrorException: setfield!: immutable struct of type Circle cannot be changed

### Mutable composite types
*Mutable* composite types are defined just like immutable types, *except* that we prepend the `struct` keyword with the keyword `mutable`, e.g.,
```julia
mutable struct typename
    field specification block
end
```

In [39]:
mutable struct CircleM
    x::Int
    y::Int
    r::Int
    R::Int
    G::Int
    B::Int
end

In [40]:
mycirc = CircleM(10,20,5,0,0,0)

CircleM(10, 20, 5, 0, 0, 0)

In [41]:
fieldnames(CircleM)

(:x, :y, :r, :R, :G, :B)

In [42]:
mycirc.R

0

In [43]:
mycirc.R = 10

10

### Parametric types
Types may take parameters, and this way create a new family of types. Example:
```julia
struct Point{T}
    x::T
    y::T
end
```
allows us to operate with structs such as `Point{Float64}`, `Point{Int}`, etc., without defining new types for the various types of the fields.

In [44]:
struct Point{T}
    x::T
    y::T
end

In [45]:
Point(1,2)    # Here, the type `T` is inferred from the arguments of 
              # the constructor.

Point{Int64}(1, 2)

In [46]:
Point(1.0,2.0)    # Here, the type `T` is inferred from the arguments of 
                  # the constructor. *Both* arguments must be of the same 
                  # type for this to work.

Point{Float64}(1.0, 2.0)

In [47]:
Point{Float64}(1,2)    # Here, the type `T` is explicitly specified 
                       # via the `{Float64}` construct

Point{Float64}(1.0, 2.0)

### Type tuples
Instead of specifying a single parametric type, we can specify one type for each field, e.g.,
```julia
struct Point1{X,Y}
    x::X
    y::Y
end
```


In [48]:
struct Point1{X,Y}
    x::X
    y::Y
end

In [49]:
Point1(1,2.)    # This time, the struct (`Point1`) is able to infer both types, 
                # since we have allowed them to differ.

Point1{Int64, Float64}(1, 2.0)

### Type unions
Sometimes, we want to consider unions of types. This can be done by `Union{T1,T2,...,Tn}` where `Tj` is some union.

In [50]:
mixedtype = Union{Int,Float32}

Union{Float32, Int64}

In [51]:
32::mixedtype, 3.2f1::mixedtype

(32, 32.0f0)

In [52]:
3.2e1::mixedtype    # 3.2e1 is of type Float64, which does not belong to
                    #  `mixedtype`

TypeError: TypeError: in typeassert, expected Union{Float32, Int64}, got a value of type Float64

## Performance tips
### Measure performance with @time
Julia has a built-in, simple *macro*  for measuring time consumption and memory allocation during execution: `@time`. There are better tools for doing this, but they require additional packages which we have not looked into yet.

The timing macro is used by prepending a statement with `@time`. If the statement goes over several lines, we can use the `@time begin ... end` construct, where the execution block is inserted instead of `...`.

Julia is based on Just-In-Time compilation, which implies that the code is compiled the first time a function us executed. Thus, timing the first execution will include both the compilation time and the execution time. It is therefore not fair to report execution of the first run of a function.

Because Julia is based on multiple dispatch, and may have many methods which vary depending on their arguments/argument types, this also has the consequence that if an argument type is changed, from one call to another, the new method that will then be used must also be compiled. In other words: changing the arguments/argument type requires a fresh compilation.

In [53]:
@time rand(5)

  0.000006 seconds (1 allocation: 96 bytes)


5-element Vector{Float64}:
 0.48912933050925445
 0.3721907149998902
 0.9577277306681533
 0.5915005992512532
 0.16358297275347522

In [54]:
@time rand(10^9);

  1.281257 seconds (2 allocations: 7.451 GiB, 0.37% gc time)


In [55]:
@time begin
    x = rand(10^8)
    y = sin.(x)
end;

  1.238581 seconds (85.15 k allocations: 1.496 GiB, 32.55% gc time, 2.63% compilation time)


### Avoid global variables
Variables defined in the REPL have global scope, i.e., they are accessible everywhere, even within `for` loops and within functions. These variables are known as "global variables", and global variable `x` can be defined in either of the following two ways (which are equivalent):
```julia
julia> x = 5
julia> global x = 5
```
A global variables may have its value, and therefore its type, changed at any point. Because of this, it is difficult for the JIT compiler to optimize the resulting code.

Global variables are often meant to be constants, and if we prepend the declaration with keyword `constant`,
```julia
julia> constant x = 5
```
we make it impossible to change the value and thus the type. Thus, when declared as a `constant`, the JIT compiler has much more possibilities to optimize the code, while still allowing the variable to have global scope (i.e., its value is available inside `for` loops and functions). The disadvantage is, of course, that we need to restart the Julia session to change the value of the constant.

Passing a value as an argument to a function is a much better way of operating in Julia, since the Julia compiler can optimize the compilation of the function for the actual type that is used.

In [56]:
x = 5
f() = x/3
f()

1.6666666666666667

In [57]:
# Calling `f()` in a loop where the global variable is used.
@time begin
    for i in 1:10^7
        f()
    end
end

  0.297789 seconds (10.00 M allocations: 152.588 MiB, 7.75% gc time)


In [58]:
g(x) = x/3
g(x)

1.6666666666666667

In [59]:
# Calling `g(x)` in a loop where the global variable is passed on as 
# input argument to the function.
#
@time begin
    for i in 1:10^7
        g(x)
    end
end

  0.211445 seconds (10.00 M allocations: 152.588 MiB, 5.92% gc time)


### Avoid changing type
Changing code, e.g., in a loop, reduces the execution speed. 

In [60]:
@time begin
    for i in 1:10^6
        x = [1,2,3]    # x is defined as a vector of Int
        for j in 1:10
            x = 1 ./(1 .+ x)    # the type of x is changed to Float64
        end
    end
end

  8.003919 seconds (51.24 M allocations: 2.027 GiB, 2.89% gc time, 1.19% compilation time)


In [61]:
@time begin
    for i in 1:10^6
        x = [1.0,2.0,3.0]    # x is defined as a vector of Float64
        for j in 1:10
            x = 1 ./(1 .+ x)    # there is no change of type for x
        end
    end
end

  7.785134 seconds (51.00 M allocations: 2.012 GiB, 2.91% gc time)


### Break functions into multiple methods
Instead of creating complex functions where there are tests on input arguments, with `if` statements to handle the various cases depending on input arguments, it is better to write one method for each case, where the argument type decides which method is used.

Example of a poor, complex function:
```julia
function f(x)
    if typeof(x) == Int
        statements
    elseif typeof(x) == Rational
        statements
    elseif typeof(x) == Float64
        statements
    end
    return result
end
```
is complex/poor code. Instead, split the function into multiple methods:
```julia
function f(x)
    statements
    return result
end
#
function f(x::Int)
    statements
    return result
end
#
function f(x::Rational)
    statements
    return result
end
```

### Write type-stable functions
Type-stable functions return results that do not depend on the input argument type/value. 

### Access arrays in column-major order
Arrays are stored in column-major order in Julia. It is quicker to access elements in contiguous memory order, so arrays should be manipulated in column-major order.

In [62]:
A = rand(0:9,2,3)

2×3 Matrix{Int64}:
 1  0  8
 7  9  6

In [63]:
A[:]    # Linear indexing reveals the the column-major ordering of arrays

6-element Vector{Int64}:
 1
 7
 0
 9
 8
 6

In [64]:
# Here, the inner loop adds unity *row-wise*
ni = 10^4
nj = 10^4
A = rand(0:9,ni,nj)
#
@time begin
    for i in 1:ni
        for j in 1:nj
            A[i,j] += 1
        end
    end
end

 13.220910 seconds (289.82 M allocations: 5.809 GiB, 3.13% gc time, 0.03% compilation time)


In [65]:
# Here, the inner loop adds unity *column-wise*, which is consistent with column-major ordering
ni = 10^4
nj = 10^4
A = rand(0:9,ni,nj)
#
@time begin
    for j in 1:nj
        for i in 1:ni
            A[i,j] += 1
        end
    end
end

 10.294266 seconds (289.82 M allocations: 5.809 GiB, 3.92% gc time)


### Reduce array allocation
It is common to introduce temporary variables in functions. Variables in functions are stored either in the so-called *stack* memory or in the so-called *heap* memory. Every time a function is called, the programming language allocates some stack memory to the function; the stack memory is relatively small, and is deleted every time the program exits the function.

When more memory is needed than is available in the stack, these variables are put into the *heap* memory. This typically happens for larger collections such as arrays, etc. The programming language does not automatically delete allocated heap memory, and this implies that new memory space is allocated every time the function is called. If a function is called many times, allocated memory in the heap can grow uncontrollably. This build-up of memory allocation is particular problematic in code where functions are called many times, such as in differential equation solvers, root solvers, optimization code, etc.

Some programming languages, such as C, have special commands for *deleting* the allocated memory in heap. If this deletion of allocated memory is not done properly, the programs are known to have a "memory leak". In other languages, such as Julia, Python, MATLAB, etc., special routines for "garbage collection" are used to reduce the growth of allocated memory.

One way to reduce this build-up of heap memory allocation is to reduce the introduction of new variables in a function. This can be done by passing a variable as an argument in the function call, and then use this input argument to store temporary variables and results. In this way, we instead operate on an existing memory structure: no matter how many times the function is called, we are still working on the same memory addresses. The "downside" of this is, of course, that the variable in the function call is *changed*. 

In [66]:
# Example where vector `x` is created every time function `f` is called.
function f()
    x = Vector{Float64}(undef,200)
    for i in 1:length(x)
        x[i] = rand()
    end
    return x
end
f();    # Calling the function once to avoid compilation in the @time macro

In [67]:
# Calling the function 10^7 times, we have 10^7 memory allocations, and 
# the loops spends a considerable amount of time on garbage collection
#
@time begin
    for i in 1:10^7
        f()
    end
end

  8.900915 seconds (10.00 M allocations: 16.838 GiB, 19.29% gc time)


In [68]:
# Example where vector `x` is passed as argument, and where we manipulate 
# the content of this "global" variable. No variables are put in the heap in 
# function `g`
#
function g(x)
    for i in 1:length(x)
        x[i] = rand()
    end
    return x
end
g(rand(2)); # Calling the function once to avoid compilation in the @time macro

In [69]:
# There is zero heap memory allocation in the loop since we operate on 
# the predefined, global variable `x` all the time. Obviously, there is 
# no need for garbage collection.
#
x = rand(100)
@time begin
    for i in 1:10^7
        g(x)
    end
end

  1.288085 seconds
